In [1]:
nb_folds = 5
seed = 2024

input_dir = "input/"
labels_dir = input_dir + "station_data"
satelite_dir = input_dir + "satelite_data"

!echo $labels_dir
!ls -alh $labels_dir | head

!echo $satelite_dir
!ls -alh $satelite_dir | tail

input/station_data
total 4.0M
drwxrwxr-x 2 moto moto 4.0K Jan 26 20:41 .
drwxrwxr-x 4 moto moto 4.0K Feb 12 11:15 ..
-rw-rw-r-- 1 moto moto 349K Aug 25 01:19 Atlantic_City_1993_2013_training_data.csv
-rw-rw-r-- 1 moto moto 329K Aug 25 01:19 Baltimore_1993_2013_training_data.csv
-rw-rw-r-- 1 moto moto 315K Aug 25 01:19 Eastport_1993_2013_training_data.csv
-rw-rw-r-- 1 moto moto 321K Aug 25 01:19 Fort_Pulaski_1993_2013_training_data.csv
-rw-rw-r-- 1 moto moto 291K Aug 25 01:19 Lewes_1993_2013_training_data.csv
-rw-rw-r-- 1 moto moto 335K Aug 25 01:19 New_London_1993_2013_training_data.csv
-rw-rw-r-- 1 moto moto 313K Aug 25 01:19 Newport_1993_2013_training_data.csv
input/satelite_data
-rw-rw-r-- 1 moto moto 241K Aug 25 21:41 dt_ena_20131222_vDT2021.nc
-rw-rw-r-- 1 moto moto 241K Aug 25 21:41 dt_ena_20131223_vDT2021.nc
-rw-rw-r-- 1 moto moto 241K Aug 25 21:41 dt_ena_20131224_vDT2021.nc
-rw-rw-r-- 1 moto moto 241K Aug 25 21:42 dt_ena_20131225_vDT2021.nc
-rw-rw-r-- 1 moto moto 241K Aug 25 21

In [2]:
# !pip install netCDF4
import netCDF4
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import glob

In [3]:
import numpy as np
import pandas as pd
import pickle
import glob

from sklearn.model_selection import StratifiedKFold, StratifiedGroupKFold
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

import numpy as np
np.bool = bool

from sklearn import metrics
import matplotlib.pyplot as plt
from tqdm import tqdm

# Mine
from GCForest import gcForest # https://github.com/pylablanche/gcForest/tree/master
import utils
utils.seed_basic(seed)

!python --version

Python 3.9.21


In [4]:
from sklearn.metrics import recall_score, precision_score, f1_score, roc_auc_score, accuracy_score
from sklearn.metrics import precision_recall_curve, auc

In [5]:
filenames = glob.glob(f"{labels_dir}/*.csv")
df_full = []
for filename in filenames:
    if "merged" in filename:
        continue
    print(filename)
    df_run = pd.read_csv(filename)
    df_full.append(df_run)

print(len(df_full))
df_full = pd.concat(df_full).reset_index(drop=True)

first_date = '1993-01-01'
first_year = int(first_date[:4])

print(df_full.groupby(["location"])["anomaly"].agg(["count", "mean", "sum"]))
df_full["fold"] = df_full["t"].apply(lambda t: (int(t[:4]) - first_year) % nb_folds)
print(df_full.groupby(["fold"])["anomaly"].agg(["count", "mean"]))
df_full.tail()

input/station_data/Fort_Pulaski_1993_2013_training_data.csv
input/station_data/Eastport_1993_2013_training_data.csv
input/station_data/Sewells_Point_1993_2013_training_data.csv
input/station_data/Lewes_1993_2013_training_data.csv
input/station_data/The_Battery_1993_2013_training_data.csv
input/station_data/New_London_1993_2013_training_data.csv
input/station_data/Portland_1993_2013_training_data.csv
input/station_data/Baltimore_1993_2013_training_data.csv
input/station_data/Newport_1993_2013_training_data.csv
input/station_data/Washington_1993_2013_training_data.csv
input/station_data/Atlantic_City_1993_2013_training_data.csv
input/station_data/Sandy_Hook_1993_2013_training_data.csv
12
               count      mean  sum
location                           
Atlantic City   7429  0.043478  323
Baltimore       7649  0.023271  178
Eastport        7495  0.062041  465
Fort Pulaski    7628  0.014552  111
Lewes           7639  0.041367  316
New London      7607  0.004470   34
Newport         7

,t,anomaly,location,latitude,longitude,fold
90443,2013-12-27,0,Sandy Hook,40.466944,-74.009444,0
90444,2013-12-28,0,Sandy Hook,40.466944,-74.009444,0
90445,2013-12-29,0,Sandy Hook,40.466944,-74.009444,0
90446,2013-12-30,0,Sandy Hook,40.466944,-74.009444,0
90447,2013-12-31,0,Sandy Hook,40.466944,-74.009444,0


In [6]:
df_full.groupby("t")["location"].count().sort_values()

t
2002-02-28     9
2004-01-25     9
2004-01-24     9
2004-01-16     9
2004-02-01     9
              ..
2000-02-17    12
2000-02-16    12
2000-02-15    12
2000-02-25    12
2013-12-31    12
Name: location, Length: 7670, dtype: int64

In [7]:
df_full[df_full["t"] == "2007-04-19"].sort_values("location")

,t,anomaly,location,latitude,longitude,fold
80449,2007-04-19,1,Atlantic City,39.356667,-74.418053,4
58113,2007-04-19,1,Baltimore,39.266944,-76.579444,4
12692,2007-04-19,1,Eastport,44.904598,-66.982903,4
5179,2007-04-19,1,Fort Pulaski,32.036700,-80.901700,4
27973,2007-04-19,1,Lewes,38.782780,-75.119164,4
42843,2007-04-19,1,New London,41.361401,-72.089996,4
65738,2007-04-19,0,Newport,41.504333,-71.326139,4
50473,2007-04-19,1,Portland,43.658060,-70.244170,4
88048,2007-04-19,1,Sandy Hook,40.466944,-74.009444,4
20332,2007-04-19,0,Sewells Point,36.946701,-76.330002,4


In [8]:
df_full["year"] = df_full["t"].apply(lambda t: int(t[:4]))
df_full.groupby("year")["anomaly"].agg(["count", "mean", "sum"]).sort_values("mean")                         

,count,mean,sum
year,,,
1994,4330,0.012933,56
2002,4255,0.013161,56
2001,4205,0.014744,62
2004,3934,0.015506,61
1999,4260,0.015962,68
2000,4352,0.016544,72
1995,4367,0.019006,83
2008,4366,0.023591,103
2007,4364,0.026123,114


In [9]:
locations = [
    "Date",
    "Average_SLA",
    "Atlantic City",
    "Baltimore",
    "Eastport",
    "Fort Pulaski",
    "Lewes",
    "New London",
    "Newport",
    "Portland",
    "Sandy Hook",
    "Sewells Point",
    "The Battery",
    "Washington",
][2:]
print(locations)

['Atlantic City', 'Baltimore', 'Eastport', 'Fort Pulaski', 'Lewes', 'New London', 'Newport', 'Portland', 'Sandy Hook', 'Sewells Point', 'The Battery', 'Washington']


In [10]:
filenames = glob.glob(f"{satelite_dir}/*.nc")
print(len(filenames))

dict_sla = {}
for filename in tqdm(filenames):
    date_str = filename.split("/")[-1].split(".")[0].split("_")[2] # dt_ena_19930101_vDT2021.nc"
    if len(date_str) != 8:
        continue
    formatted_date = f"{date_str[:4]}-{date_str[4:6]}-{date_str[6:8]}"

    nc_filename = filename
    
    dataset = netCDF4.Dataset(nc_filename, mode="r")
    sla = dataset.variables["sla"][:]
    sla = np.ma.getdata(sla[0, :, :].filled(0))
    dict_sla[formatted_date] = sla

    dataset.close()
len(dict_sla)

7302


100%|██████████████████████████████████████| 7302/7302 [01:03<00:00, 114.32it/s]


7302

In [11]:
dates = list(dict_sla.keys())
dates.sort()
print(len(dates), dates[:3], dates[-3:])

7302 ['1993-01-01', '1993-01-02', '1993-01-03'] ['2013-12-29', '2013-12-30', '2013-12-31']


In [12]:
cities_coordinates = {
    "Atlantic City": {"latitude": 39.3643, "longitude": -74.4229},
    "Baltimore": {"latitude": 39.2904, "longitude": -76.6122},
    "Eastport": {"latitude": 44.9034, "longitude": -66.9895},
    "Fort Pulaski": {"latitude": 32.0317, "longitude": -80.8970},
    "Lewes": {"latitude": 38.7746, "longitude": -75.1394},
    "New London": {"latitude": 41.3557, "longitude": -72.0995},
    "Newport": {"latitude": 41.4901, "longitude": -71.3128},
    "Portland": {"latitude": 43.6591, "longitude": -70.2568},
    "Sandy Hook": {"latitude": 40.4649, "longitude": -74.0054},
    "Sewells Point": {"latitude": 36.9463, "longitude": -76.3297},
    "The Battery": {"latitude": 40.7033, "longitude": -74.0170},
    "Washington": {"latitude": 38.9072, "longitude": -77.0369},
}

# 100 160
cities_indexes = {
    'Atlantic City': {'latitude': (55, 60), 'longitude': (100, 105)},
    'Baltimore': {'latitude': (55, 60), 'longitude': (91, 96)},
    'Eastport': {'latitude': (77, 82), 'longitude': (130, 135)},
    'Fort Pulaski': {'latitude': (26, 31), 'longitude': (74, 79)},
    'Lewes': {'latitude': (53, 58), 'longitude': (97, 102)},
    'New London': {'latitude': (63, 68), 'longitude': (109, 114)},
    'Newport': {'latitude': (63, 68), 'longitude': (112, 117)},
    'Portland': {'latitude': (72, 77), 'longitude': (116, 121)},
    'Sandy Hook': {'latitude': (59, 64), 'longitude': (101, 106)},
    'Sewells Point': {'latitude': (45, 50), 'longitude': (92, 97)},
    'The Battery': {'latitude': (60, 65), 'longitude': (101, 106)},
    'Washington': {'latitude': (53, 58), 'longitude': (89, 94)}
}

def get_features_row(row, verbose=False, margin = 3):
    sla = dict_sla[row["t"]]
    indexes = cities_indexes[row["location"]]
    latitude = indexes["latitude"]
    longitude = indexes["longitude"]

    sla_row = sla[latitude[0] - margin : latitude[1] + margin, longitude[0] - margin : longitude[1] + margin]
    if verbose:
        print(sla.shape, sla_row.shape)
        print(sla.shape)

    return sla_row

get_features_row(df_full.iloc[0], verbose=True)

(100, 160) (11, 11)
(100, 160)


array([[ 0.    ,  0.    ,  0.    , -0.0195, -0.0245, -0.0204, -0.0118,
         0.    ,  0.0127,  0.0235,  0.0292],
       [ 0.    ,  0.    , -0.0173, -0.0157, -0.0197, -0.0251, -0.0305,
        -0.025 , -0.0001,  0.029 ,  0.045 ],
       [ 0.    ,  0.    ,  0.    , -0.019 , -0.0162, -0.0268, -0.0362,
        -0.0282, -0.0114,  0.016 ,  0.038 ],
       [ 0.    ,  0.    ,  0.    , -0.0142, -0.0128, -0.0228, -0.036 ,
        -0.0271, -0.0184, -0.0039, -0.0034],
       [ 0.    ,  0.    ,  0.    ,  0.    , -0.0039, -0.0206, -0.0248,
        -0.0315, -0.0382, -0.0372, -0.033 ],
       [ 0.    ,  0.    ,  0.    ,  0.    ,  0.    , -0.0259, -0.0245,
        -0.0213, -0.0208, -0.0418, -0.0426],
       [ 0.    ,  0.    ,  0.    ,  0.    ,  0.    , -0.0031,  0.0283,
        -0.0062, -0.0147, -0.0205, -0.0213],
       [ 0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,
        -0.0028, -0.0072, -0.0016,  0.0103],
       [ 0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,
 

In [13]:
def sampling(df, target_ratio=0.02, target_col="anomaly"):
    if df[target_col].mean() >= target_ratio:
        return df.copy()
    
    df_pos = df[df[target_col] == 1].copy()
    df_neg = df[df[target_col] != 1].copy().sample(frac=1, random_state=seed).reset_index(drop=True)

    nb_neg = int( (df_pos.shape[0])*(1-target_ratio) / target_ratio)
    df_com = pd.concat([df_pos, df_neg.head(nb_neg)]).sample(frac=1, random_state=seed).reset_index(drop=True)
    return df_com

df_com = sampling(df_full)
print(df_com["anomaly"].value_counts())
df_com["anomaly"].value_counts(normalize=True)

anomaly
0    87950
1     2498
Name: count, dtype: int64


anomaly
0    0.972382
1    0.027618
Name: proportion, dtype: float64

In [ ]:
city = locations[3]
fold = 0
dict_models = {}
target_col = "anomaly"
ignored_cities = ["Newport"]

# if True:
run_folds = [-1] # list(range(nb_folds)) + [-1]
print("run_folds:", run_folds)
run_margins = [3, 5, 7, 10, 15, 20] # list(range(nb_folds)) + [-1]
print("run_margins:", run_margins)

for margin in run_margins:
    print("="*40)
    dict_models[margin] = {}
    
    for city in locations:
        print("="*40)
        dict_models[margin][city] = {}
        
        df_folds = df_full[df_full["location"] == city]
        df_folds = df_folds[df_folds["t"].isin(dates)].sort_values("t").reset_index(drop=True)
        print(city, df_folds.shape, df_folds[target_col].mean())
    
        for fold in run_folds:
            dict_models[margin][city][fold] = {}
            
            print("="*40)
            print( f"Processing margin {margin} city {city}  fold {fold} ... ")
            
            df_fold_train = df_folds[df_folds["fold"] != fold].reset_index(drop=True)
            if fold >= 0:
                df_fold_val = df_folds[df_folds["fold"] == fold].reset_index(drop=True)
            else:
                df_fold_val = df_fold_train.copy()

            # Only sampling the train set
            # df_fold_train = sampling(df_fold_train).sort_values("t").reset_index(drop=True)
            
            X_train = []
            for index, row in tqdm(df_fold_train.iterrows()):
                img = get_features_row(row, margin=margin)
                X_train.append(img.flatten())
            X_train = np.stack(X_train)
            
            X_val = []
            for index, row in tqdm(df_fold_val.iterrows()):
                img = get_features_row(row, margin=margin)
                X_val.append(img.flatten())
            X_val = np.stack(X_val)
            
            y_train = np.array(df_fold_train[target_col])
            y_val = np.array(df_fold_val[target_col])
            
            print(X_train.shape, X_val.shape, y_train.mean(), y_val.mean())
        
            # model = CascadeForestClassifier(random_state=1)
            model = gcForest(shape_1X=[8,8], window=[4,6], tolerance=0.0, min_samples_mgs=10, min_samples_cascade=7)
            model.fit(X_train, y_train) # Be careful when fold==-1, train == val
            
            y_pred = model.predict_proba(X_val)
            y_pred_bin = (y_pred[:,1] >= 0.5).astype(int)
            
            acc = accuracy_score(y_val, y_pred_bin)
            
            if (y_pred_bin.mean() == 0) or (y_pred_bin.mean() == 1):
                auc = -1
                f1 = -1
            else:
                auc = roc_auc_score(y_val, y_pred[:,1])
                f1 = f1_score(y_val, y_pred[:,1] >= 0.5, pos_label=1)
        
            print(f"F1: {f1}, ACC: {acc}, AUC: {auc}")
            
            dict_models[margin][city][fold] = {
                "model": model,
                "metrics": {
                    "acc": acc,
                    "auc": auc,
                    "f1": f1,
                },
                "y_pred": y_pred,
                "y_val": y_val,
                "df_val": df_fold_val
            }
            print("="*40)
        
        len(dict_models)

run_folds: [-1]
run_margins: [3, 5, 7, 10, 15, 20]
Atlantic City (7063, 7) 0.04289961772617868
Processing margin 3 city Atlantic City  fold -1 ... 


7063it [00:00, 7742.01it/s]
7063it [00:00, 7848.18it/s]


(7063, 121) (7063, 121) 0.04289961772617868 0.04289961772617868


/home/moto/miniconda3/envs/py39/lib/python3.9/site-packages/sklearn/ensemble/_forest.py:615: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable OOB estimates.
  warn(


Adding/Training Layer, n_layer=1
Layer validation accuracy = 0.9518754423213022
Adding/Training Layer, n_layer=2
Layer validation accuracy = 0.9532908704883227
Adding/Training Layer, n_layer=3
Layer validation accuracy = 0.9525831564048125
F1: 0.9064748201438849, ACC: 0.9926376893671245, AUC: 0.9999492256917999
Baltimore (7286, 7) 0.023744166895415867
Processing margin 3 city Baltimore  fold -1 ... 


...

(7063, 361) (7063, 361) 0.04289961772617868 0.04289961772617868


/home/moto/miniconda3/envs/py39/lib/python3.9/site-packages/sklearn/ensemble/_forest.py:615: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable OOB estimates.
  warn(


Adding/Training Layer, n_layer=1
Layer validation accuracy = 0.9568294409058741
Adding/Training Layer, n_layer=2
Layer validation accuracy = 0.9568294409058741
F1: 0.9983471074380166, ACC: 0.9998584171032139, AUC: 0.9999995117854981
Baltimore (7286, 7) 0.023744166895415867
Processing margin 7 city Baltimore  fold -1 ... 


7286it [00:00, 7891.17it/s]
7286it [00:00, 7836.87it/s]


(7286, 361) (7286, 361) 0.023744166895415867 0.023744166895415867


In [ ]:
def get_best_threshold(y_pred, y_val, verbose=False):
    # Get the probability values for the positive class
    y_prob = y_pred[:, 1]
    y_test = y_val[:]
    
    # Initialize the variables to store the best threshold and F1 score
    best_threshold = 0
    best_f1 = 0
     
    # Loop through the possible threshold values from 0 to 1 with a step of 0.01
    for threshold in np.arange(0, 1, 0.01):
        # Apply the threshold to the probability values to get the predicted labels
        y_pred_bin = (y_prob >= threshold).astype(int)
        if (y_pred_bin.mean() > 0) and (y_pred_bin.mean() < 1):
            # Calculate the F1 score for the current threshold
            f1 = f1_score(y_test, y_pred_bin)
            # Update the best threshold and F1 score if the current F1 score is higher
            if f1 > best_f1:
                best_threshold = threshold
                best_f1 = f1

    auc = -1
    if (y_val.mean() > 0) and (y_val.mean() < 1):
        auc = roc_auc_score(y_val, y_pred[:,1])
    
    if verbose:
        # Print the best threshold and F1 score
        print(f"The optimal threshold is {best_threshold:.2f}")
        print(f"The F1 score using the optimal threshold is {best_f1:.3f}")

    return float(best_threshold), float(round(best_f1, 3)), float(round(auc, 3))

In [ ]:
dict_meta = {}

for margin in run_margins:
    f1_full = []
    for city in locations:
        for fold in run_folds:
            meta_run = dict_models[margin][city][fold]
            y_pred = meta_run["y_pred"]
            y_val = meta_run["y_val"]
            best_threshold, best_f1, auc = get_best_threshold(y_pred, y_val)
            
            # print( city, margin, fold, "=>", best_threshold, best_f1)

            if (city not in dict_meta) or (best_f1 > dict_meta[city]["params"]["best_f1"]):
                dict_meta[city] = {
                    "model": meta_run["model"],
                    "params": {
                        "margin": margin,
                        "best_f1": best_f1,
                        "best_threshold": best_threshold,
                        "auc": auc
                    }
                }
                
for city in locations:
    print(dict_meta[city]["params"])

In [ ]:
# dict_params = {}
# for city in dict_meta:
#     dict_params[city] = dict_meta[city]["params"]
# dict_params
dict_params = {'Atlantic City': {'margin': 3,
  'best_f1': 0.223,
  'best_threshold': 0.11,
  'auc': 0.622},
 'Baltimore': {'margin': 7,
  'best_f1': 0.214,
  'best_threshold': 0.27,
  'auc': 0.693},
 'Eastport': {'margin': 3,
  'best_f1': 0.15,
  'best_threshold': 0.06,
  'auc': 0.535},
 'Fort Pulaski': {'margin': 3,
  'best_f1': 0.078,
  'best_threshold': 0.05,
  'auc': 0.704},
 'Lewes': {'margin': 5,
  'best_f1': 0.256,
  'best_threshold': 0.24,
  'auc': 0.706},
 'New London': {'margin': 7,
  'best_f1': 0.118,
  'best_threshold': 0.07,
  'auc': 0.632},
 'Newport': {'margin': 3, 'best_f1': 0.0, 'best_threshold': 0.0, 'auc': -1.0},
 'Portland': {'margin': 3,
  'best_f1': 0.105,
  'best_threshold': 0.22,
  'auc': 0.588},
 'Sandy Hook': {'margin': 7,
  'best_f1': 0.199,
  'best_threshold': 0.15,
  'auc': 0.624},
 'Sewells Point': {'margin': 5,
  'best_f1': 0.368,
  'best_threshold': 0.08,
  'auc': 0.764},
 'The Battery': {'margin': 5,
  'best_f1': 0.087,
  'best_threshold': 0.03,
  'auc': 0.614},
 'Washington': {'margin': 10,
  'best_f1': 0.307,
  'best_threshold': 0.2,
  'auc': 0.68}}

In [ ]:
dict_meta = {}

for city in locations:
    margin = dict_params[city]["margin"]
    fold = -1
    meta_run = dict_models[margin][city][fold]

    dict_meta[city] = {
        "model": meta_run["model"],
        "params": dict_params[city]
    }

for city in locations:
    print(dict_meta[city]["params"])

In [ ]:
sub_dir = "subs/sub_supervision"
with open(f'{sub_dir}/dict_meta.pickle', 'wb') as handle:
    pickle.dump(dict_meta, handle)

In [ ]:
model.predict_proba([img.flatten()])

In [ ]:
prob = model.predict_proba([img.flatten()])
prob

# Combination

In [ ]:
# Custom models
with open(os.path.join("subs/sub_isolation", "dict_meta.pickle"), "rb") as f:
    dict_ano = pickle.load(f)
dict_ano

In [ ]:
dict_bin = dict_meta

In [ ]:
dict_com = {}
for city in locations:
    if dict_bin[city]["params"]["best_f1"] >= dict_ano[city]["params"]["best_f1"]:
        dict_com[city] = dict_bin[city]
    else:
        dict_com[city] = dict_ano[city]
len(dict_com)

In [ ]:
sub_dir = "subs/sub_combination"
!mkdir -p $sub_dir
with open(f'{sub_dir}/dict_meta.pickle', 'wb') as handle:
    pickle.dump(dict_com, handle)

In [ ]:
dict_com